In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/russian-language-toxic-comments/labeled.csv


In [3]:
from sklearn.model_selection import train_test_split

file_path = '/kaggle/input/russian-language-toxic-comments/labeled.csv'
df = pd.read_csv(file_path)

print(df.head())

X = df.drop('toxic', axis=1)  
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f'Размер обучающей выборки: {X_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}')


                                             comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0
Размер обучающей выборки: (12970, 1)
Размер тестовой выборки: (1442, 1)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


print(f'Количество строк в данных: {df.shape[0]}')
print(f'Столбцы данных: {df.columns.tolist()}')

X = df['comment']  
y = df['toxic'].apply(int)

print(f'Количество пропущенных значений в X: {X.isnull().sum()}')
print(f'Количество пропущенных значений в y: {y.isnull().sum()}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f'Размер обучающей выборки: {X_train.shape[0]}')
print(f'Размер тестовой выборки: {X_test.shape[0]}')

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f'Размеры X_train_tfidf: {X_train_tfidf.shape}')
print(f'Размеры X_test_tfidf: {X_test_tfidf.shape}')

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_tfidf, y_train)
y_pred_log_reg = log_reg.predict(X_test_tfidf)
f1_log_reg = f1_score(y_test, y_pred_log_reg)
print(f'F1-score для Логистической регрессии: {f1_log_reg:.4f}')

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train_tfidf, y_train)
y_pred_tree_clf = tree_clf.predict(X_test_tfidf)
f1_tree_clf = f1_score(y_test, y_pred_tree_clf)
print(f'F1-score для Дерева решений: {f1_tree_clf:.4f}')


Количество строк в данных: 14412
Столбцы данных: ['comment', 'toxic']
Количество пропущенных значений в X: 0
Количество пропущенных значений в y: 0
Размер обучающей выборки: 12970
Размер тестовой выборки: 1442
Размеры X_train_tfidf: (12970, 63804)
Размеры X_test_tfidf: (1442, 63804)
F1-score для Логистической регрессии: 0.6773
F1-score для Дерева решений: 0.5784


In [7]:
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred)
print(f'F1-score для TF-IDF + Logistic Regression: {f1:.4f}')

F1-score для TF-IDF + Logistic Regression: 0.6870


In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from datasets import Dataset
import torch
from torch.nn import CrossEntropyLoss

# Загрузка и предобработка данных
file_path = '/kaggle/input/russian-language-toxic-comments/labeled.csv'
df = pd.read_csv(file_path)
X = df['comment']
y = df['toxic'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train

4194     Парадоксальная ситуация. У меня все лампы что ...
8396     Работодатель может поднимать вакансии и эта да...
2798       15 и повышение в должности. Отлично работаем!\n
5958     Русачки ебаные, какого хуя вы отдали Польше мо...
8523     играться то че не так, судя по фото видео они ...
                               ...                        
5191     Ну не знаю. А откуда тогда машин столько у люд...
13418    А у меня кандидатка завалилась на электрохроми...
5390                          Второй сезон полная хуета!\n
860      Мда... Хотел просто спасибо сказать, но и тут ...
7270     Не, год это прямо максимум максимум. В среднем...
Name: comment, Length: 12970, dtype: object

In [ ]:
# Настройка модели и токенизатора
model_name = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [11]:
# Задаем dropout для уменьшения переобучения
model.config.hidden_dropout_prob = 0.3

# Подготовка данных для модели
train_data = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train}))
test_data = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test}))

In [12]:
# Функция токенизации с уменьшенной длиной для экономии памяти
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/12970 [00:00<?, ? examples/s]

Map:   0%|          | 0/1442 [00:00<?, ? examples/s]

In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    logging_steps=10,
    report_to="none"  
)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    f1 = f1_score(labels, pred)
    return {'f1': f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

trainer.train()

eval_result = trainer.evaluate()
f1_rubert_tiny2 = eval_result['eval_f1']
print(f'F1-score для rubert-tiny2: {f1_rubert_tiny2:.4f}')

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1
1,0.128500,0.194455,0.890052
2,0.142000,0.211035,0.902542
3,0.155300,0.239447,0.895680


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


F1-score для rubert-tiny2: 0.9025
